In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

import time
import torch.nn as nn
import matplotlib.pyplot as plt
import cv2 
import itertools

import csv
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
#from vgg16_2 import VGG_16

from vgg16_2 import VGG_16_2


import argparse
from data_loader_half_2 import get_loader
from data_loader_half_2 import get_dataset

import math
import torchvision.models as models
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import time
import shutil
import os
import torch.nn.functional as F

# Loading dataset

In [4]:
celeba_image_dir = '/home/name/celeba/stargan/data/celeba/images'
attr_path = '/home/name/celeba/stargan/data/celeba/list_attr_celeba.txt'
id_path = '/home/name/celeba/stargan/data/celeba/identity_CelebA.txt'
selected_attrs = ['Black_Hair']
celeba_crop_size = 224#178
image_size = 224 #128
batch_size = 40
num_workers = 0
mode1 = 'train' #test or train
mode2 = 'test'
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu") #"cuda:0" if torch.cuda.is_available() else 
torch.set_default_tensor_type('torch.cuda.FloatTensor')
 
print(device)

cuda:2


In [5]:
celeba_loader_train, data_train = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode1, num_workers)
celeba_loader_test, data_test = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode2, num_workers)

Finished preprocessing the CelebA dataset...
Finished preprocessing the CelebA dataset...


In [6]:
print(len(data_train))
print(len(data_test))

88899
12700


In [7]:

model = VGG_16_2()


# Validate

In [8]:
def validate_single_label(model):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels[:,0].to(device)
        labels = labels.long()
        preds = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        print("________")
        size += len(labels)
        del labels
        
    print(size)
    print(correct)
    print(float(correct)/float(size))

# Train

In [9]:
def train(model, criterion, optimizer):
    model.to(device)
    model.train()
    size = 0
    correcta = 0
    for i, (im, labels) in enumerate(celeba_loader_train):
        #if (i ==1):
         #   print (labels)
        im, labels = im.to(device), labels[:,0].to(device)
        print(labels.shape)
        labels = labels.long()
        optimizer.zero_grad()
        output = model(im)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        print (loss, "loss")
        #preds = model(im)
        values, indices = output.max(-1)
        size += len(labels)
        correcta += indices.eq(labels).sum()
        correct = indices.eq(labels).sum()
        del im
        del labels
        del output
        del values
        del indices
        
        print(correct, "correct out of 40")
    print(correcta, "correct out of all")
    print(size)
    print(float(correcta)/float(size))
    return correct, size
        #print(list(filter(lambda p: p.requires_grad, model.parameters()))[0][0])
        



In [13]:
criterion = nn.CrossEntropyLoss().to(device)#nn.NLLLoss().to(device) #95% 
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=0.00002)

In [15]:

lr =  0.0000001 #6 ep, 4 01 , 
for epoch in range (2): # real
   
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=0)
    cor, size = train(model, criterion, optimizer)
    print (cor,size)
 
    #validate_single_label(model)
    
    #wd helps
    if (epoch>  5):
        lr  = 0.00001
        print(lr)

torch.Size([40])
tensor(0.1614, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(39, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.4391, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3027, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(36, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2840, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3731, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2528, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(37, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3532, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2038, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(37, device='cuda:2') co

tensor(0.2685, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2897, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2007, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(39, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3625, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(29, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3801, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2170, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(36, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3550, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(31, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2789, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
t

tensor(0.5063, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3813, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.4161, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3208, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3008, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3034, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2475, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(36, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2925, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
t

tensor(0.2958, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3504, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.4631, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(30, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3796, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2799, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2803, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(36, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3068, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3384, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(30, device='cuda:2') correct out of 40
t

tensor(0.2680, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3316, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3542, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.4694, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(31, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3652, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2669, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2458, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(38, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3780, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
t

...

tensor(0.4263, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3626, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3656, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.1784, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(37, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2983, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3136, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3256, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(35, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3470, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
t

tensor(0.2410, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3198, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3887, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(31, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.4023, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.4502, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3495, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3802, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2796, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
t

tensor(0.4294, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(31, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2986, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3787, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.4030, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3193, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.1970, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(37, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3313, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(32, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2995, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
t

tensor(0.3647, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3666, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.3195, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2980, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(33, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2464, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(38, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.2887, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.6491, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(27, device='cuda:2') correct out of 40
torch.Size([40])
tensor(0.4553, device='cuda:2', grad_fn=<NllLossBackward>) loss
tensor(34, device='cuda:2') correct out of 40
t

KeyboardInterrupt: 

In [16]:
validate_single_label(model) #90.60

________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
________
_